# ETL Pipeline Preparation

In this project, a ML classification algorithm is trained with +27.000 messages contained in `messages.cvs`. On the other hand, the categories to which messages must be assigned are contained in a separate file called `categories.csv`. Hence, in the ETL stage, we must merge both `messages` and `categories`, then clean the merged dataset and finally save the data ready for ML in a database.

This Notebook is to prepare and set up all ETL steps so to, once they are ready, put them in a python script with the entire pipeline (see final cell of this notebook). 

### 1. Import libraries and load datasets.

In [1]:
# import libraries
import pandas as pd
import sqlite3

Let's first load and quickly explore `messages` dataset

In [2]:
# load 'messages' dataset:
messages = pd.read_csv('messages.csv', index_col='id')
messages.head()

,message,original,genre
id,,,
2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [66]:
# quick full view of 1st 20 messages
messages['message'].to_list()[0:20]

['Weather update - a cold front from Cuba that could pass over Haiti',
 'Is the Hurricane over or is it not over',
 'Looking for someone but no name',
 'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.',
 'says: west side of Haiti, rest of the country today and tonight',
 'Information about the National Palace-',
 'Storm at sacred heart of jesus',
 'Please, we need tents and water. We are in Silo, Thank you!',
 'I would like to receive the messages, thank you',
 'I am in Croix-des-Bouquets. We have health issues. They ( workers ) are in Santo 15. ( an area in Croix-des-Bouquets )',
 "There's nothing to eat and water, we starving and thirsty.",
 'I am in Petionville. I need more information regarding 4636',
 'I am in Thomassin number 32, in the area named Pyron. I would like to have some water. Thank God we are fine, but we desperately need water. Thanks',
 "Let's do it together, need food in Delma 75, in didine area",
 'More informati

In [67]:
# Quick inspection of the column 'genre' and its possible values
messages['genre'].value_counts()

news      13068
direct    10782
social     2398
Name: genre, dtype: int64

In [68]:
# First 10 messages with genre = 'direct'
messages.loc[messages['genre']=='direct']['message'].to_list()[0:10]

['Weather update - a cold front from Cuba that could pass over Haiti',
 'Is the Hurricane over or is it not over',
 'Looking for someone but no name',
 'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.',
 'says: west side of Haiti, rest of the country today and tonight',
 'Information about the National Palace-',
 'Storm at sacred heart of jesus',
 'Please, we need tents and water. We are in Silo, Thank you!',
 'I would like to receive the messages, thank you',
 'I am in Croix-des-Bouquets. We have health issues. They ( workers ) are in Santo 15. ( an area in Croix-des-Bouquets )']

In [69]:
# First 10 messages with genre = 'news'
messages.loc[messages['genre']=='news']['message'].to_list()[0:5]

['UNICEF is also upgrading solid waste management systems in seven rural schools that currently serve as shelters.',
 'China periodically faces spills into rivers that result in water supplies being cut off, most seriously in 2005 when an explosion at an industrial plant sent toxic chemicals streaming into the Songhua River further upstream, in Harbin.',
 "DATO' SRI MOHD NAJIB BIN TUN HAJI ABDUL RAZAK, Prime Minister of Malaysia, said that five years ago he had called for a global movement of moderates of all religions and of all countries to marginalize extremists and shape an agenda towards peace and pragmatism.",
 'This is a vital step in ensuring respect for human rights and strengthening democracy.',
 "The pumping of contaminated water from the basement floor of Unit 1's turbine building into its main condenser is in progress, whereas at Unit 2 that process has not begun because the steam condenser is full."]

In [70]:
# First 10 messages with genre = 'social'
messages.loc[messages['genre']=='social']['message'].to_list()[0:10]

['My thoughts and prayers go out to all the lives that were affected today by the earthquake in Haiti. I will keep you all in my prayers!',
 'I m sorry for the poor people in Haiti tonight. That being said I d love to experience an earthquake just once. In a big field of course.',
 'RT selenagomez UNICEF has just announced an emergency alert for the people of Haiti who were hit by a 7.0 earthquake and a tsunami.. ..',
 'lilithia yes 5.2 magnitude earthquake hit manila philippines last night.',
 'RT TheNewsBlotter RT caribnews On Call International opens #Haiti earthquake emergency hotline for those with families friends in r ..',
 'Most Eureka homeowners won&#39 t collect on quake insurance The California Earthquake Authority does not expec. http bit.ly 7RY2qT',
 'Haiti hit by largest earthquake in over 200 years AP AP The strongest earthquake in more than 200 years .. http ow.ly 16kgyd',
 'RT pinkelephantpun Earthquake Relief Donate http bit.ly 67t0d0 Select Immediate Needs to help th

Now let's explore the other dataset: `categories`

In [71]:
# Let's now load 'categories' dataset
categories = pd.read_csv('categories.csv', index_col='id')
categories.head()

,categories
id,
2,related-1;request-0;offer-0;aid_related-0;medi...
7,related-1;request-0;offer-0;aid_related-1;medi...
8,related-1;request-0;offer-0;aid_related-0;medi...
9,related-1;request-1;offer-0;aid_related-1;medi...
12,related-1;request-0;offer-0;aid_related-0;medi...


As seen above, both `categories` and `messages` datasets have an 'id' column with which, we are instructed, both datasets can be merged to obtain a single dataset containing features and target variables together. To confirm that this is the case, and that both datasets can be actually merged, let's check that they both have the same exact size:

In [72]:
print(len(messages),len(categories))

26248 26248


Let's see non-index columns and their different values in both datasets to have an overview of what ETL steps we will need. Indeed, the categories dataset looks messy with too many unique values:

In [73]:
messages.nunique()

message     26177
original     9630
genre           3
dtype: int64

In [74]:
categories.nunique()

categories    4003
dtype: int64

### 2. Merge datasets.
Let's start by merging both datasets and assigning them to `df` which will be cleaned in the following steps:

In [75]:
# merge datasets
df = messages.merge(categories, how='left', on='id')
df.head()

,message,original,genre,categories
id,,,,
2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


In [76]:
df.shape

(26386, 4)

### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.


As seen above, the 'categories' column in `df` shows if the message corresponds to one or several categories (numerically encoding yes/no as 1/0), and all those values are concatenated in a single cell value. Let's then split all those values into separate columns, one column for each category. We'll do this in the original `categories` dataset and then later re-merge it with `df`.

In [77]:
# let's create a dataframe of the 36 individual category columns
categories = categories['categories'].str.split(';',expand=True)
categories.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
id,,,,,,,,,,,,,,,,,,,,,
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
7,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
8,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
9,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
12,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


We now need to rename those newly created columns. To do this, we can use the first row of `categories` to create column names for the categories data. Then, we can rename columns of `categories` with new column names.

In [78]:
# select the first row of the categories dataframe
first_row = categories.iloc[0]
first_row
# Extract a list of new column names for categories, by slicing up to the last 2 characters in each cell
category_colnames = [x[:-2] for x in first_row]
print(category_colnames)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [79]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
7,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
8,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
9,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
12,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 4. Convert category values to just numbers 0 or 1.

Now we need to extract the actual category number value and remove the category column name from each cell. For example, `related-0` becomes `0`, `related-1` becomes `1`. To do so, we can slice the last value of the cell string and then convert it to number

In [80]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = categories[column].str[-1:]
    # convert column from string to numeric
    categories[column] = pd.to_numeric(categories[column])
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5. Replace `categories` column in `df` with new category columns.
Now we will Drop the 'categories' column from the `df` dataframe, since it is no longer needed, and then concatenate `df` and `categories` data frames.

In [81]:
df.drop(columns=['categories'], inplace=True)
df.head()

,message,original,genre
id,,,
2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [82]:
# Re-checking that both dataframes can be concatenated as they have the same exact number of rows
print(df.shape, categories.shape)

(26386, 3) (26248, 36)


In [83]:
# concatenate 'df' and 'categories'
df = df.merge(categories, how='left', on='id')
df.head()

,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
df.shape

(26668, 39)

### 6. Remove duplicates.
Now let's check how many duplicates are in this dataset, then drop the duplicates and then confirm that duplicates were removed.

In [85]:
# check number of duplicates
df.nunique()

message                   26177
original                   9630
genre                         3
related                       3
request                       2
offer                         2
aid_related                   2
medical_help                  2
medical_products              2
search_and_rescue             2
security                      2
military                      2
child_alone                   1
water                         2
food                          2
shelter                       2
clothing                      2
money                         2
missing_people                2
refugees                      2
death                         2
other_aid                     2
infrastructure_related        2
transport                     2
buildings                     2
electricity                   2
tools                         2
hospitals                     2
shops                         2
aid_centers                   2
other_infrastructure          2
weather_

In [86]:
n_rows_before_removing_duplicates = len(df)
n_rows_before_removing_duplicates

26668

We see above that `df`has more rows than unique values in the column 'message'. Thus, let's drop duplicates considering the 'message' column:

In [87]:
# drop duplicates
df.drop_duplicates(subset=['message'], inplace=True)

In [88]:
# check number of duplicates
df.nunique()

message                   26177
original                   9630
genre                         3
related                       3
request                       2
offer                         2
aid_related                   2
medical_help                  2
medical_products              2
search_and_rescue             2
security                      2
military                      2
child_alone                   1
water                         2
food                          2
shelter                       2
clothing                      2
money                         2
missing_people                2
refugees                      2
death                         2
other_aid                     2
infrastructure_related        2
transport                     2
buildings                     2
electricity                   2
tools                         2
hospitals                     2
shops                         2
aid_centers                   2
other_infrastructure          2
weather_

In [89]:
n_rows_after_removing_duplicates = len(df)

In [90]:
print("Number of rows before removing duplicates = {}\nNumber of rows after removing duplicates = {}\n{} rows were removed".format(n_rows_before_removing_duplicates, len(df), n_rows_before_removing_duplicates- len(df)))

Number of rows before removing duplicates = 26668
Number of rows after removing duplicates = 26177
491 rows were removed


Now we have the same number of rows than the number of unique messages (26177)

However, there are a couple of things to cleanup. First, the column 'related' has 3 possible values, which is weird. Let's check them and see if we can process it so to have the column with only 2 possible values (0 or 1)

In [49]:
df['related'].value_counts()

1    19874
0     6115
2      188
Name: related, dtype: int64

Let's print the first 10 messages of each value of 'related' to see if we can figure out the meaning of the 3 levels of 'related'

In [50]:
# column 'related' = 0
df.loc[df['related']==0]['message'].to_list()[0:10]

['Information about the National Palace-',
 'I would like to receive the messages, thank you',
 'I am in Petionville. I need more information regarding 4636',
 'are you going to call me or do you want me to call ou? let me know?',
 "I don't understand how to use this thing 4636.",
 'Can you tell me about this service',
 'Good evening, Radio one please. I would like information on Tiyous.',
 "I'm here, I didn't find the person that I needed to send the pant by phone",
 "I'm listening to you at Miraguan we asking the government to take change because one gallon gas is 80.",
 'i am very happy, i hear god, religious hyme']

In [51]:
# column 'related' = 1
df.loc[df['related']==1]['message'].to_list()[0:10]

['Weather update - a cold front from Cuba that could pass over Haiti',
 'Is the Hurricane over or is it not over',
 'Looking for someone but no name',
 'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.',
 'says: west side of Haiti, rest of the country today and tonight',
 'Storm at sacred heart of jesus',
 'Please, we need tents and water. We are in Silo, Thank you!',
 'I am in Croix-des-Bouquets. We have health issues. They ( workers ) are in Santo 15. ( an area in Croix-des-Bouquets )',
 "There's nothing to eat and water, we starving and thirsty.",
 'I am in Thomassin number 32, in the area named Pyron. I would like to have some water. Thank God we are fine, but we desperately need water. Thanks']

In [52]:
# column 'related' = 2
df.loc[df['related']==2]['message'].to_list()[0:15]

['Dans la zone de Saint Etienne la route de Jacmel est bloqu, il est trsdifficile de se rendre  Jacmel',
 '. .. i with limited means. Certain patients come from the capital.',
 "The internet caf Net@le that's by the Dal road by the Maranata church ( incomplete )",
 "Bonsoir, on est a bon repos aprs la compagnie teleko sur la route a droite de l'impasse Roger colas aprs la 9e maison sur la main droite de la rue, on est environ 30 personnes sur un. ..",
 'URGENT CRECHE ORPHANAGE KAY TOUT TIMOUN CROIX DES MISSIONS IMPASSE BALEV BUTTE BOYER MANQUE EAU ET NOURRITURE N ONT VU AUCUN SECOURS DEPUIS 8 JOURS HELP HELP',
 "elle est vraiment malade et a besoin d'aide. utilisez mon numero de tlphone pour obtenir plus de renseignements. Nous attendons une reponse. Aucun numero fourni par contre.",
 "no authority has passed by to see us. We don't have a place t sleep ( incomplete )",
 "It's Over in Gressier. The population in the area - Incomplete",
 'we sleep with the baby. Thanks in advance for the

Thus, column 'related' means if it is or not related with the catastrophe. Related = 0 -> unrelated, related = 1 -> related. Related = 2 refers to incomplete or untranslated messages that are likely to be related but in some cases it is uncertain without the English Translation. Those values = 2 can probably be removed if needed later on.

Let's look now at the column 'child' which only has one value

In [53]:
df['child_alone'].value_counts()

0    26177
Name: child_alone, dtype: int64

In [54]:
df.loc[df['child_alone']==0]['message'].to_list()[0:10]

['Weather update - a cold front from Cuba that could pass over Haiti',
 'Is the Hurricane over or is it not over',
 'Looking for someone but no name',
 'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.',
 'says: west side of Haiti, rest of the country today and tonight',
 'Information about the National Palace-',
 'Storm at sacred heart of jesus',
 'Please, we need tents and water. We are in Silo, Thank you!',
 'I would like to receive the messages, thank you',
 'I am in Croix-des-Bouquets. We have health issues. They ( workers ) are in Santo 15. ( an area in Croix-des-Bouquets )']

Values in this column clearly do not reflect its intended meaning, thus I will entirely remove the column 'child_alone' from the dataset

In [55]:
df.drop(columns=['child_alone'], inplace = True)
df.nunique()

message                   26177
original                   9630
genre                         3
related                       3
request                       2
offer                         2
aid_related                   2
medical_help                  2
medical_products              2
search_and_rescue             2
security                      2
military                      2
water                         2
food                          2
shelter                       2
clothing                      2
money                         2
missing_people                2
refugees                      2
death                         2
other_aid                     2
infrastructure_related        2
transport                     2
buildings                     2
electricity                   2
tools                         2
hospitals                     2
shops                         2
aid_centers                   2
other_infrastructure          2
weather_related               2
floods  

In [56]:
df.shape

(26177, 38)

Now we have a dataframe with all category columns having 2 possible values and the same number of unique values in the 'message' column as rows in the dataframe.

### 7. Save the clean dataset into an sqlite database.
Let's use pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library

In [79]:
conn = sqlite3.connect('DisasterResponse.db')
df.to_sql("messages_w_categories", conn, if_exists="replace", index=False)

26177

In [80]:
# Testing that the database has been created
test = pd.read_sql('select * from messages_w_categories', conn)
test.head()

,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 8. Use this notebook to complete a pipeline python script

All the steps in this notebook have been placed in a python script called `process_data.py`, which is found in the folder 'data', together with a testing file with unit tests called `test_process_data.py`

In [60]:
# Test process_data.py script
conn2 = sqlite3.connect(r'C:\Users\jvios\Downloads\Data Science nanodegree (Udacity)\Course 3_Data Engineering for Data Scientists\4.- Project_Disaster response support via automated text message classification\IDE\data\database.db')
test2 = pd.read_sql('select * from messages_w_categories', conn2)
test2.head()

,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
